In [ ]:
import pandas as pd

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.tree import DecisionTreeClassifier, plot_tree

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import seaborn as sns

In [ ]:
try:
    df = pd.read_csv('iris.csv')
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print("Error: 'iris.csv' not found. Please ensure the file is in the correct directory.")
    exit()
